<a id="top"></a>

<a id = '1.0'></a>
<h1 style = "font-family: garamond; font-size: 40px; font-style: normal;background-color: #2ab7ca; color : #fed766; border-radius: 5px 5px;padding:5px;text-align:center; font-weight: bold" >Quick Navigation</h1>

    
* [Dependencies and Configuration](#1)
* [Stage 4: Modelling](#2)
    * [How EDA helped us?](#31)
    * [Modelling](#31)
        * [Spot Checking Algorithms](#31)
            * [Make Basic Pipeline (Say No to Data Leakage!)](#31)
            * [Define Metrics](#31)
            * [Comparison of Cross-Validated Models](#31)
            * [Out-of-Fold Confusion Matrix](#31)
            * [Hypothesis Testing Across Models](#31)
        * [Model Selection: Hyperparameter Tuning with GridSearchCV](#31)
        * [Retrain on the whole training set](#31)
            * [Retrain using Optimal Hyperparameters](#31)
        * [Interpretation of Results](#31)
            * [Interpretation of Coefficients](#31)
            * [Interpretation of Metric Scores on Train Set](#31)
    * [Evaluation on Test Set](#31)
    * [Bias-Variance Tradeoff](#31)

# Dependencies and Configuration

In [29]:
import csv
import random
from functools import wraps
from time import time
from typing import Callable, Dict, List, Union, Optional, Any

import matplotlib.pyplot as plt
import copy
import mlxtend
import numpy as np
import pandas as pd
import seaborn as sns
from mlxtend.evaluate import paired_ttest_5x2cv, bias_variance_decomp
from scipy import stats
from sklearn import (base, decomposition, dummy, ensemble, feature_selection,
                     linear_model, metrics, model_selection, neighbors,
                     pipeline, preprocessing, svm, tree)

from statsmodels.regression.linear_model import OLS
#from statsmodels.stats.outliers_influence import variance_inflation_factor

from dataclasses import dataclass
import logging

In [2]:
@dataclass
class config:
    raw_data: str = "https://storage.googleapis.com/reighns/reighns_ml_projects/docs/supervised_learning/regression/%20house-sales-in-king-country-usa/data/raw/kc_house_data.csv"
    train_size: float = 0.8
    seed: int = 1992
    num_folds: int = 5
    cv_schema: str = "KFold"

In [3]:
def set_seeds(seed: int = 1234) -> None:
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    
def init_logger(log_file: str = "info.log"):
    """
    Initialize logger.
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s", datefmt= "%Y-%m-%d,%H:%M:%S"))
    file_handler = logging.FileHandler(filename=log_file)
    file_handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s",  datefmt= "%Y-%m-%d,%H:%M:%S"))
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    return logger

In [4]:
# set seeding for reproducibility
_ = set_seeds(seed = config.seed)

# set logger
logger = init_logger()

# read data
df = pd.read_csv(config.raw_data)

# How EDA helped us?

<div class="alert alert-success" role="alert">
    <b>Insights derived from EDA:</b> 
    <li> To fill in.
</div>

# Cross-Validation Strategy

<div class="alert alert-block alert-danger">
<b>Generalization:</b>     
    <blockquote cite="https://www.huxley.net/bnw/four.html">
        <p>Ultimately, we are interested in the Generalization Error made by the model, that is, how well the model perform on <b>unseen data</b> that is not taken from our sample set $\mathcal{D}$. In general, we use <b>validation set</b> for <b>Model Selection</b> and the <b>test set</b> for <b>an estimate of generalization error</b> on new data.
            <br> <b>- Refactored from Elements of Statistical Learning, Chapter 7.2</b></p>
    </blockquote>
</div>

---

<div class="alert alert-success" role="alert">
    <b>Step 1: Train-Test-Split:</b> Since this dataset is relatively small, we will not use the <b>train-validation-test</b> split and only split into train and test in a ratio of 9:1, whereby the split is stratified on our target, using <code>stratify=y</code> parameter in <code>train_test_split()</code> to ensure that our target has equal representation in both train and test. We note that this is a relatively small dataset and in practice, we need a large sample size to get a reliable/stable split, it is also recommended to retrain the whole dataset (without the "unseen" test set) after we have done the model selection process (eg. finding best hyperparameters). 
</div>

---

<div class="alert alert-success" role="alert">
    <b>Step 2: Resampling Stategy:</b> Note that we will be performing <code>StratifiedKFold</code> as our resampling strategy. After our split in Step 1, we have a training set $X_{\text{train}}$, we will then perform our resampling strategy on this $X_{\text{train}}$. We will choose our choice of $K = 5$. The choice of $K$ is somewhat arbitrary, and is derived <a href="https://stats.stackexchange.com/questions/61783/bias-and-variance-in-leave-one-out-vs-k-fold-cross-validation">empirically</a>. 
</div>

---

To recap, we have the following:

- **Training Set ($X_{\text{train}}$)**: This will be further split into K validation sets during our cross-validation. This set is used to fit a particular hypothesis $h \in \mathcal{H}$.
- **Validation Set ($X_{\text{val}}$)**: This is split from our $X_{\text{train}}$ during cross-validation. This set is used for model selection (i.e. find best hyperparameters, attempt to produce a best hypothesis $g \in \mathcal{H}$).
- **Test Set ($X_{\text{test}}$)**: This is an unseen test set, and we will only use it after we finish tuning our model/hypothesis. Suppose we have a final best model $g$, we will use $g$ to predict on the test set to get an estimate of the generalization error (also called out-of-sample error).

---

<figure>
<img src='https://scikit-learn.org/stable/_images/grid_search_workflow.png' width="500"/>
<figcaption align = "center"><b>Courtesy of scikit-learn on a typical Cross-Validation workflow.</b></figcaption>
</figure>

In [5]:
predictor_cols = df.columns[3:].tolist()
target_col = ["price"]
logger.info(f"\nThe predictor columns are \n{predictor_cols}")

2021-11-10,12:09:44 - 
The predictor columns are 
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [6]:
X = df[predictor_cols].copy()
y = df[target_col].copy()

In [7]:
# Split train - test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, train_size=config.train_size, shuffle=True, random_state=config.seed
)

logger.info(f"\nShape of train: {X_train.shape}\nShape of test: {X_test.shape}")

2021-11-10,12:09:44 - 
Shape of train: (17290, 18)
Shape of test: (4323, 18)


In [8]:
def make_folds(
    df: pd.DataFrame,
    num_folds: int,
    cv_schema: str,
    seed: int,
    predictor_col: List,
    target_col: List,
) -> pd.DataFrame:
    """Split the given dataframe into training folds.

    Args:
        df (pd.DataFrame): [description]
        num_folds (int): [description]
        cv_schema (str): [description]
        seed (int): [description]

    Returns:
        pd.DataFrame: [description]
    """

    if cv_schema == "KFold":
        df_folds = df.copy()
        kf = model_selection.KFold(n_splits=num_folds, shuffle=True, random_state=seed)

        for fold, (train_idx, val_idx) in enumerate(
            kf.split(X=df_folds[predictor_col], y=df_folds[target_col])
        ):
            df_folds.loc[val_idx, "fold"] = int(fold + 1)

        df_folds["fold"] = df_folds["fold"].astype(int)

    elif cv_schema == "StratifiedKFold":
        df_folds = df.copy()
        skf = model_selection.StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)

        for fold, (train_idx, val_idx) in enumerate(
            skf.split(X=df_folds[predictor_col], y=df_folds[target_col])
        ):
            df_folds.loc[val_idx, "fold"] = int(fold + 1)

        df_folds["fold"] = df_folds["fold"].astype(int)
        print(df_folds.groupby(["fold", "diagnosis"]).size())

    return df_folds

In [9]:
X_y_train = pd.concat([X_train, y_train], axis = 1).reset_index(drop=True)
df_folds = make_folds(X_y_train, num_folds=config.num_folds, cv_schema=config.cv_schema, seed=config.seed, predictor_col= predictor_cols, target_col = target_col)

Looks good! All our five folds are now in `df_fold`!

# Modelling

## Spot Checking Algorithms

<div class="alert alert-success" role="alert">
    <b>Terminology Alert!</b> This method is advocated by <a href="https://machinelearningmastery.com/">Jason Brownlee PhD</a> and this serves as the first stage of my modelling process. We will rapidly test (spot check) different classifier algorithms, from <code>DummyClassifier</code>, to <code>LinearModel</code> to more sophisticated ensemble trees like <code>RandomForest</code>. 
</div>

---

I also note to the readers that we need to think of a few things when choosing the "optimal" machine learning algorithm:

- [No Lunch Free Theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem) intuitively says that no single optimization algorithm can work best in all situations. Therefore, spot checking can help us form a basis of which algorithm might work better in this particular scenario.
- [Occam's Razor](https://en.wikipedia.org/wiki/Occam%27s_razor) often appears in many Machine Learning textbook, and the narrative is that a simpler model more often times generalizes better than a complex model. This is not unfamiliar when we think of the bias-variance tradeoff, and that is why there is always a tradeoff that we must make.

### Make Basic Pipeline (Say No to Data Leakage!)

<div class="alert alert-block alert-danger">
<b>Say No to Data Leakage:</b> This has been emphasized throughout and we must be careful as we should never touch the test set when fitting the model.
    <li> This means that preprocessing steps such as <code>StandardScaling()</code> should only be fitted on the training data, and then apply the same transformation (mean and std) on the test data. In other words, do not apply scaling on the whole dataset before splitting. 
    <li> However, it is also equally important to take note <b>not to contaminate</b> our validation set, which is often overlooked, resulting in over optimistic results from model selection phase, but perform badly on unseen test set. As a result, when we use a 5 fold cross validation, we should be careful during fitting that the preprocessing steps are only applied on the training folds, and not on all 5 folds.
    <li> The same idea is also applied to our <code>ReduceVIF()</code> preprocessing step. We should also include this in our pipeline and not select the features outside the cross-validation loop.</li>
</div>   
    
Scikit Learn's `Pipeline` object will prevent us from data leakage, as the steps in a pipeline is already pre-defined. There is also a lot of flexibility in this object, as you can even write custom functions in your pipeline!

In [10]:
def variance_inflation_factor(exog, idx_kept, vif_idx):
    """Compute VIF for one feature.
    
    Args:
        exog (np.ndarray): Observations
        idx_kept (List[int]): Indices of features to consider
        vif_idx (int): Index of feature for which to compute VIF
    
    Returns:
        float: VIF for the selected feature
    """
    exog = np.asarray(exog)
    
    x_i = exog[:, vif_idx]
    mask = [col for col in idx_kept if col != vif_idx]
    x_noti = exog[:, mask]
    
    r_squared_i = OLS(x_i, x_noti).fit().rsquared
    vif = 1. / (1. - r_squared_i)
    
    return vif

class ReduceVIF(base.BaseEstimator, base.TransformerMixin):
    """The base of the class structure is implemented in https://www.kaggle.com/ffisegydd/sklearn-multicollinearity-class;
    I heavily modified the class such that it can take in numpy arrays and correctly implemented the fit and transform method.
    """

    def __init__(self, thresh=10, max_drop=20):
        self.thresh = thresh
        self.max_drop = max_drop
        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def reset(self):
        """Resets the state of predictor columns after each fold."""

        self.column_indices_kept_ = []
        self.feature_names_kept_ = None

    def fit(self, X, y=None):
        """Fits the Recursive VIF on the training folds and save the selected feature names in self.feature_names

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """
        
        self.column_indices_kept_, self.feature_names_kept_ = self.calculate_vif(X)     
        
        return self

    def transform(self, X, y=None):
        """Transforms the Validation Set according to the selected feature names.

        Args:
            X ([type]): [description]
            y ([type], optional): [description]. Defaults to None.

        Returns:
            [type]: [description]
        """

        return X[:, self.column_indices_kept_]

    def calculate_vif(self, X: Union[np.ndarray, pd.DataFrame]):
        """Implements a VIF function that recursively eliminates features.

        Args:
            X (Union[np.ndarray, pd.DataFrame]): [description]

        Returns:
            [type]: [description]
        """
        feature_names = None
        column_indices_kept = list(range(X.shape[1]))
        
        if isinstance(X, pd.DataFrame):
            feature_names = X.columns

        dropped = True
        count = 0
        
        while dropped and count <= self.max_drop:
            dropped = False
            
            max_vif, max_vif_col = None, None
            
            for col in column_indices_kept:
                
                vif = variance_inflation_factor(X, column_indices_kept, col)
                
                if max_vif is None or vif > max_vif:
                    max_vif = vif
                    max_vif_col = col
            
            if max_vif > self.thresh:
                print(f"Dropping {max_vif_col} with vif={max_vif}")
                column_indices_kept.remove(max_vif_col)
                
                if feature_names is not None:
                    feature_names.pop(max_vif_col)
                    
                dropped = True
                count += 1
                
        return column_indices_kept, feature_names

In [11]:
# create a feature preparation pipeline for a model
def make_pipeline(model):
    """Make a Pipeline for Training.

    Args:
        model ([type]): [description]

    Returns:
        [type]: [description]
    """
    
    steps = list()
    # standardization
    steps.append(('standardize', preprocessing.StandardScaler()))
    # reduce VIF
    # steps.append(("remove_multicollinearity", ReduceVIF(thresh=10)))
    # the model
    steps.append(('model', model))
    # create pipeline
    _pipeline = pipeline.Pipeline(steps=steps)
    return _pipeline

In [12]:
regressors = [
    # baseline model
    dummy.DummyRegressor(strategy="mean"),
    # linear model
    linear_model.LinearRegression(fit_intercept=True),
    linear_model.Ridge(random_state=config.seed, alpha=1, fit_intercept=True),
    linear_model.Lasso(
        random_state=config.seed,
        alpha=1,
        fit_intercept=True,
    ),
    linear_model.ElasticNet(
        random_state=config.seed,
        alpha=1,
        l1_ratio=0.5,
        fit_intercept=True,
    ),
    # tree
    tree.DecisionTreeRegressor(
        random_state=config.seed, criterion="squared_error"
    ),
    # ensemble
    #  ensemble.RandomForestClassifier(random_state=config.seed),
]

In [13]:
regressors = [make_pipeline(model) for model in regressors]

### Define Metrics

In [14]:
default_result_names = [
    "y_true",
    "y_pred",
]

default_logit_names = [
    "y_true",
    "y_pred",
]

default_score_names = [
    "explained_variance_score",
    "mean_squared_error",
    "mean_absolute_error",
    "root_mean_squared_error",
    "r2_score",
    "mean_absolute_percentage_error",
]

custom_score_names = ["adjusted_r2"]


def adjusted_r2(r2: float, n: int, k: int) -> float:
    """Calculate adjusted R^2.

    Args:
        r2 (float): r2 score of the model/
        n (int): number of samples.
        k (int): number of features minus the constant bias term.

    Returns:
        adjusted_r2_score (float): r2 * (n - 1) / (n - k - 1)
    """
    adjusted_r2_score = r2 * (n - 1) / (n - k - 1)
    return adjusted_r2_score

In [15]:
class Results:
    """Stores results for model training in columnwise format."""
    
    _result_dict: Dict
        
    logit_names: List[str]
    score_names: List[str]
        
    def __init__(
        self,
        logit_names: List[str] = default_logit_names,
        score_names: List[str] = default_score_names,
        existing_dict: Optional[Dict] = None,
    ):
        """Construct a new results store."""       
        self.logit_names = logit_names
        self.score_names = score_names
        
        if existing_dict is not None:
            self._result_dict = copy.deepcopy(existing_dict)
            return
        
        dict_keys = ["identifier", *logit_names, *score_names]
        
        self._result_dict = {
            key: [] for key in dict_keys
        }
    
    def add(self, identifier: str, results: Dict, in_place=False):
        """Add a new results row."""        
        if not in_place:
            return Results(
                self.logit_names,
                self.score_names,
                self._result_dict
            ).add(identifier, results, in_place=True)
        
        self._result_dict["identifier"].append(identifier)
        
        for result_name in set([*results.keys(), *self.logit_names, *self.score_names]):
            
            result_value = results.get(result_name, np.nan)
            
            self._result_dict[result_name].append(result_value)
        
        return self
    
    def get_result(self, result_name: str) -> Dict[str, Any]:
        """Get a map of identifiers to result values for a result."""
        return {
            identifier: result_value for
            identifier, result_value in
            zip(self._result_dict["identifier"], self._result_dict[result_name])
        }
    
    def get_result_values(self, result_name: str) -> List[Any]:
        """Get a list of values for a result."""
        return self._result_dict[result_name]
    
    def to_dataframe(self) -> pd.DataFrame:
        """Get a Data Frame containing the results."""
        return pd.DataFrame.from_dict(self._result_dict)
    
    def to_dict(self) -> Dict:
        """Get a dictionary containing the results.
        
        Returns:
             Dict[str, List[Any]]: Dictionary of result columns 
        """
        return self._result_dict

In [17]:
def compute_metrics(logits: Dict[str, np.ndarray]) -> Dict[str, Any]:
    """Compute metrics from logits."""

    y_true, y_pred = logits["y_true"], logits["y_pred"]

    default_score_names = [
        "explained_variance_score",
        "mean_squared_error",
        "mean_absolute_error",
        "root_mean_squared_error",
        "r2_score",
        "mean_absolute_percentage_error",
    ]

    default_metrics_dict: Dict[str, float] = {}
    custom_metrics_dict: Dict[str, float] = {}

    for metric_name in default_score_names:
        if hasattr(metrics._regression, metric_name):
            # TODO: get metric score with default parameters, consider adding kwargs if you want to configure parameters
            metric_score = getattr(metrics._regression, metric_name)(y_true, y_pred)
        else:
            # logger.info(f"{metrics._regression} has no such attribute {metric_name}!")
            # add custom metrics here
            rmse = metrics._regression.mean_squared_error(y_true, y_pred, squared=False)
            custom_metrics_dict["root_mean_squared_error"] = rmse
            
        if metric_name not in default_metrics_dict:
            default_metrics_dict[metric_name] = metric_score
        
        metrics_dict = {**default_metrics_dict, **custom_metrics_dict}
            
    return metrics_dict

In [27]:
def mean_score(score_values) -> Union[float, np.ndarray]:
    """Compute the mean score."""
    
    score_values = np.array(score_values)
    
    shape = score_values.shape
    
    if len(shape) == 1:
        return score_values.mean()
    
    return score_values.mean(axis=0)

def mean_cv_results(model_results: Results) -> Dict:
    """Add mean cross-validation results.
    
    This method computes the mean value for all
    score types in the model_results, including
    for scores (e.g., confusion matrix) where
    the mean value may contain decimal places.
    """
    cv_logits = {
        y_result: np.concatenate(model_results.get_result_values(y_result))
        for y_result in
        model_results.logit_names
    }
    
    cv_scores = {
        score: mean_score(
            model_results.get_result_values(score)
        )
        for score in model_results.score_names
    }
    
    return {
        **cv_logits,
        **cv_scores,
    }

def oof_cv_results(model_results: Results) -> Dict:
    """Add OOF cross-validation results."""
    
    cv_logits = {
        y_result: np.concatenate(
            model_results.get_result_values(y_result)
        )
        for y_result in
        model_results.logit_names
    }
    
    cv_scores = compute_metrics(cv_logits)
    
    return {
        **cv_logits,
        **cv_scores,
    }

def add_cv_results(model_results: Results):
    """Add cross-validation results.
    
    This method returns a copy of the given model results
    with summary columns for mean and CV cross-validation.
    """
    mean_cv = mean_cv_results(model_results)
    oof_cv = oof_cv_results(model_results)
    
    return (
        model_results
        .add("mean_cv", mean_cv)
        .add("oof_cv", oof_cv)
    )

In [18]:
def train_on_fold(
    df_folds: pd.DataFrame,
    models: List[Callable],
    num_folds: int,
    predictor_col: List,
    target_col: List,
) -> Dict[str, List]:
    """Take in a dataframe with fold number as column, and a models which holds a list of callable models, we will loop through and return a dictionary of cv results.

    Args:
        df_folds (pd.DataFrame): [description]
        model (Callable): [description]
        num_folds (int): [description]
        predictor_col (List): [description]
        target_col (List): [description]


    Returns:
        Dict[str, List]: [description]
    """
  
    y_true = df_folds[target_col].values.flatten()

    # test_pred_arr: np.ndarray = np.zeros(len(X_test))

    model_dict = {}

    for model in models:
        model_results = Results()

        if isinstance(model, pipeline.Pipeline):
            model_name = model["model"].__class__.__name__
        else:
            model_name = model.__class__.__name__

        # out-of-fold validation predictions
        oof_pred_arr: np.ndarray = np.zeros(len(df_folds))
      
        for fold in range(1, num_folds + 1):

            train_df = df_folds[df_folds["fold"] != fold].reset_index(drop=True)
            val_df = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
            val_idx = df_folds[df_folds["fold"] == fold].index.values
            X_train, y_train = train_df[predictor_col].values, train_df[target_col].values
            X_val, y_val = val_df[predictor_col].values, val_df[target_col].values
    
            model.fit(X_train, y_train)
            y_val_pred = model.predict(X_val)

            
            logits = {
                "y_true": y_val,
                "y_pred": y_val_pred,
            }
            
            metrics = compute_metrics(logits)
            
            model_results.add(f"fold {fold}", {
                **logits,
                **metrics
            }, in_place=True)
            
           
        if model_name not in model_dict:
            model_dict[model_name] = model_results

    return model_dict

In [19]:
model_dict = train_on_fold(
    df_folds,
    models = regressors,
    num_folds=5,
    predictor_col=predictor_cols,
    target_col = target_col
)

2021-11-10,12:09:44 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:09:44 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:09:44 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:09:44 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:09:44 - <module 'sklearn.metrics._regression' from 'C:\\Us

In [30]:
model_dict_with_summary = {
    model: add_cv_results(model_results)
    for model, model_results in model_dict.items()
}

2021-11-10,12:19:12 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:19:12 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:19:12 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:19:12 - <module 'sklearn.metrics._regression' from 'C:\\Users\\reighns\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\sklearn\\metrics\\_regression.py'> has no such attribute root_mean_squared_error!
2021-11-10,12:19:12 - <module 'sklearn.metrics._regression' from 'C:\\Us

In [31]:
results_df = pd.concat({
    name: results.to_dataframe().T
    for name, results
    in model_dict_with_summary.items()
}, axis=0)

results_df.columns = ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5', 'mean_cv', 'oof_cv']
results_df

fold 1  \
DummyRegressor        identifier                                                                 fold 1   
                      y_true                          [[945000.0], [352500.0], [560000.0], [500000.0...   
                      y_pred                          [541073.7741469058, 541073.7741469058, 541073....   
                      explained_variance_score                                                     -0.0   
                      mean_squared_error                                             132247512137.06163   
                      mean_absolute_error                                                 229404.508989   
                      root_mean_squared_error                                             363658.510332   
                      r2_score                                                                -0.000407   
                      mean_absolute_percentage_error                                           0.542257   
LinearRegression      identifier                                                                 fold 1   
                      y_true                          [[945000.0], [352500.0], [560000.0], [500000.0...   
                      y_pred                          [[667876.5428875468], [614410.8399372244], [73...   
                      explained_variance_score                                                 0.680095   
                      mean_squared_error                                             42290033057.306816   
                      mean_absolute_error                                                 127943.982537   
                      root_mean_squared_error                                             205645.406118   
                      r2_score                                                                  0.68009   
                      mean_absolute_percentage_error                                           0.265815   
Ridge                 identifier                                                                 fold 1   
                      y_true                          [[945000.0], [352500.0], [560000.0], [500000.0...   
                      y_pred                          [[667923.954879363], [614423.8176756387], [731...   
                      explained_variance_score                                                 0.680109   
                      mean_squared_error                                             42288266590.795799   
                      mean_absolute_error                                                 127934.770004   
                      root_mean_squared_error                                              205641.11114   
                      r2_score                                                                 0.680104   
                      mean_absolute_percentage_error                                           0.265796   
Lasso                 identifier                                                                 fold 1   
                      y_true                          [[945000.0], [352500.0], [560000.0], [500000.0...   
                      y_pred                          [667918.069528075, 614415.1560338883, 731173.9...   
                      explained_variance_score                                                 0.680106   
                      mean_squared_error                                             42288598880.705284   
                      mean_absolute_error                                                 127937.742862   
                      root_mean_squared_error                                             205641.919075   
                      r2_score                                                                 0.680101   
                      mean_absolute_percentage_error                                           0.265806   
ElasticNet            identifier                                                                 fold 1   
                      y_true   

### Comparison of Cross-Validated Models

The point of the following comparison is to check how different models are performing across folds. More specifically, if we have 5 folds, we will have a metric score for each fold, subsequently, we can find the standard error of model's performance. We need to be aware of models that have high variance across folds in terms of the metrics performance. This can indicate that the model is highly unstable, and may be a sign of overfitting.

In [ ]:
def summarize_metrics(metric_name):
    ls = []
    for model_name, inner_dict in model_dict.items():
        folds = inner_dict["identifier"][:-2]
        all_obs = []
        for idx, obs in enumerate(inner_dict[metric_name][:-2]):
            ls.append((model_name, folds[idx], obs))
            all_obs.append(obs)
        ls.append((model_name, "SE", np.std(all_obs, ddof=1) / len(all_obs) ** 0.5))
    
    fig, ax = plt.subplots(figsize=(15, 8))
    
    summary_df = pd.DataFrame(ls, columns=["model", "fold", metric_name])
    # summary_df.to_csv
    _ = sns.boxplot(x="model", y=metric_name, data=summary_df[(summary_df['model'] != 'DummyClassifier') & (summary_df['fold'] != 'SE')], ax=ax)
    
    fig.savefig(config.spot_checking_boxplot, format='png', dpi=300)
    
    return summary_df

In [ ]:
summary_df = summarize_metrics("roc")
display(summary_df.tail(12))

### Out-of-Fold Confusion Matrix

We do have information on the performance of each folds, we now look at the performance of all 5 folds together. Typicall there are two ways to do it, one is to simply take the average of the score of five folds, the other is to take a look at out of folds predictions.

---

From the confusion matrix of the out of fold performance, Logistic Regression does seem to be a model we can explore on, although slightly lower in terms of overall AUROC score than SVC, it seems to have the quite low False Negatives amongst all. With further hyperparameter tuning and threshold optimization, we can make it better.

In [ ]:
model_names = [model for model in model_dict.keys()]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 10))

for axes, algo in zip(ax.ravel(), model_names):

    cf_mat = results_df.oof_cv[algo].confusion_matrix

    #### scores
    auc = results_df.oof_cv[algo].roc

    #### annotations
    labels = ["True Neg", "False Pos", "False Neg", "True Pos"]
    counts = ["{0:0.0f}".format(value) for value in cf_mat.flatten()]
    percentages = ["{0:.2%}".format(value) for value in cf_mat.flatten() / np.sum(cf_mat)]

    #### final annotations
    label = (
        np.array([f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(labels, counts, percentages)])
    ).reshape(2, 2)

    # heatmap
    sns.heatmap(
        data=cf_mat,
        vmin=0,
        vmax=330,
        cmap=["#fe4a49", "#2ab7ca", "#fed766", "#59981A"],
        linewidth=2,
        linecolor="white",
        square=True,
        ax=axes,
        annot=label,
        fmt="",
        cbar=False,
        annot_kws={"size": 10, "color": "black", "weight": "bold", "alpha": 0.8},
        alpha=1,
    )

    axes.text(0, -0, "{}".format(algo), {"size": 12, "color": "black", "weight": "bold"})

    axes.scatter(1, 1, s=3500, c="white")
    axes.text(
        0.72,
        1.0,
        "AUC: {}".format(round(auc, 3)),
        {"size": 10, "color": "black", "weight": "bold"},
    )

    ## ticks and labels
    axes.set_xticklabels("")
    axes.set_yticklabels("")


## titles and text
fig.text(0, 1.05, "Out Of Fold Confusion Matrix", {"size": 22, "weight": "bold"}, alpha=1)
fig.text(
    0,
    1,
    """This Visualization show the results of various classifiers and there respective
results.""",
    {"size": 14, "weight": "normal"},
    alpha=0.98,
)


fig.tight_layout(pad=2.5, w_pad=2.5, h_pad=2.5)
fig.savefig(config.oof_confusion_matrix, format='png', dpi=300)

### Hypothesis Testing Across Models

I am slightly shocked at the performance of plain LogisticRegression, I decide to use an idea from [Hypothesis Testing Across Models](http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_kfold_cv/) to check if the difference is really by chance or not. Note that I will be modifying his code as his code does not split using StratifiedKFold.

---

The basic idea is to test if two model's difference in scores (in this case roc), is statistically significant or not. However, we note that this method may violate an assumption of Student's t test.

---

- Null Hypothesis $H_0$: The difference in the performance score of two classifiers is Statistically Significant.
- Alternate Hypothesis $H_1$: The difference in the performance score of two classifiers is **not** Statistically Significant.

In [ ]:
def paired_ttest_skfold_cv(
    estimator1, estimator2, X, y, cv=10, scoring=None, shuffle=False, random_seed=None
):
    """Modified from https://github.com/rasbt/mlxtend/blob/master/mlxtend/evaluate/ttest.py to accomodate StratifiedKFold"""

    if not shuffle:
        skf = model_selection.StratifiedKFold(n_splits=cv, shuffle=shuffle)
    else:
        skf = model_selection.StratifiedKFold(
            n_splits=cv, random_state=random_seed, shuffle=shuffle
        )

    if scoring is None:
        if estimator1._estimator_type == "classifier":
            scoring = "accuracy"
        elif estimator1._estimator_type == "regressor":
            scoring = "r2"
        else:
            raise AttributeError("Estimator must " "be a Classifier or Regressor.")
    if isinstance(scoring, str):
        scorer = metrics.get_scorer(scoring)
    else:
        scorer = scoring

    score_diff = []

    for train_index, test_index in skf.split(X=X, y=y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        estimator1.fit(X_train, y_train)
        estimator2.fit(X_train, y_train)

        est1_score = scorer(estimator1, X_test, y_test)
        est2_score = scorer(estimator2, X_test, y_test)
        score_diff.append(est1_score - est2_score)

    avg_diff = np.mean(score_diff)

    numerator = avg_diff * np.sqrt(cv)
    denominator = np.sqrt(sum([(diff - avg_diff) ** 2 for diff in score_diff]) / (cv - 1))
    t_stat = numerator / denominator

    pvalue = stats.t.sf(np.abs(t_stat), cv - 1) * 2.0
    return float(t_stat), float(pvalue)


In [ ]:
# check if difference between algorithms is real
X_tmp = X_y_train[predictor_cols].values
y_tmp = X_y_train['diagnosis'].values

t, p = paired_ttest_skfold_cv(estimator1=classifiers[1], estimator2=classifiers[-1],shuffle=True,cv=5, X=X_tmp, y=y_tmp, scoring='roc_auc', random_seed=config.seed)

In [ ]:
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))

Since P value is quite high, and more the basic threshold of 0.05 or 0.1, we fail to reject the null hypothesis, and say that there is no significant difference between these two models.

## Model Selection: Hyperparameter Tuning with GridSearchCV

<div class="alert alert-success" role="alert">
    <b>Hyperparameter Tuning:</b>
    <li> We have done a quick spot checking on algorithms and realized that <code>LogisticRegression</code> is doing well for this task. For this purpose, I will just perform hyperparameter tuning on this single algorithm. However, in practice and if resources are allowed, I will also tune other models such as <code>RandomForest()</code>, or gradient boosting algorithms such as <code>XGBoost</code>, as I believe they will perform no worse than our Logistic Regression model given the right hyperparameters.
</div>

---

<div class="alert alert-info" role="alert">
    <b>Grid Search:</b>
    <li> We will use an old-fashioned way to search for hyperparameters, which is brute force method. The time complexity of Grid Search is high and if you have many hyperparameters to tune, I recommend trying out <b>Random Grid Search</b> or libraries like <b>Optuna</b> that uses Bayesian Optimization.
</div>

In [ ]:
def make_finetuning_pipeline(model):
    """Make a Pipeline for Training.

    Args:
        model ([type]): [description]

    Returns:
        [type]: [description]
    """
    
    steps = list()
    # standardization
    steps.append(('standardize', preprocessing.StandardScaler()))
    # reduce VIF
    steps.append(('remove_multicollinearity', ReduceVIF(thresh=10)))
    # the model
    steps.append(('model', model))
    # create pipeline
    _pipeline = pipeline.Pipeline(steps=steps)
    return _pipeline

Reconstruct our pipeline but now only taking in `LogisticRegression`.

In [ ]:
pipeline_logistic = make_finetuning_pipeline(
    linear_model.LogisticRegression(
        solver="saga", random_state=config.seed, max_iter=10000, n_jobs=None, fit_intercept=True
    )
)

Define our search space for the hyperparameters:

```python
param_grid = {model__penalty=["l1", "l2"],
              model__C=np.logspace(-4, 4, 10)}
```

In [ ]:
param_grid = dict(
    model__penalty=["l1", "l2"],
    model__C=np.logspace(-4, 4, 10),
)

Run our hyperparameter search with cross-validation. For example, our `param_grid` has $2 \times 10 = 20$ combinations, and our cross validation has 5 folds, then there will be a total of 100 fits.

---

Below details the pseudo code of what happens under the hood:

- Define $G$ as the set of combination of hyperparamters. Define number of splits to be $K$.
- For each set of hyperparameter $z \in Z$:
    - for fold $j$ in K:
        - Set $F_{\text{train}}=\bigcup\limits_{i\neq k}^{K} F_{i}$
        - Set $F_{\text{val}} = F_{j}$ as the validation set
        - Perform Standard Scaling on $F_{\text{train}}$ and find the mean and std
        - Perform VIF recursively on $F_{\text{train}}$ and find the selected features
        - Transform $F_{\text{val}}$ using the mean and std found using $F_{\text{train}}$
        - Transform $F_{\text{val}}$ to have only the selected features from $F_{\text{train}}$
        - Train and fit on $F_{\text{train}}$ 
    - Evaluate the fitted parameters on $F_{\text{val}}$ to obtain $\mathcal{M}$


In [ ]:
grid = model_selection.GridSearchCV(pipeline_logistic, param_grid=param_grid, cv=5, refit=True, verbose=3, scoring = "roc_auc")
_ = grid.fit(X_train, y_train)

We can save our results in a dataframe, we will also look at the top performing hyperparameter by querying the below:

```python
grid_cv_df = pd.DataFrame(grid.cv_results_)
grid_cv_df.loc[grid_cv_df['rank_test_score']==1]
```

In [ ]:
grid_cv_df = pd.DataFrame(grid.cv_results_)
best_cv = grid_cv_df.loc[grid_cv_df['rank_test_score']==1]
display(best_cv)

best_hyperparams = grid.best_params_
print(f"Best Hyperparameters found is {best_hyperparams}")

Our best performing set of hyperparameters `{'model__C': 0.3593813663804626, 'model__penalty': 'l2'}` gives rise to a mean cross validation score of $0.988739$, which is higher than the model with default hyperparameter scoring, $0.987136$.

<div class="alert alert-success" role="warning">
    <b>Room for Improvement:</b> Apart from the other methods to search for the optimal hyperparameters, we can also include preprocessing step as a tunable hyperparameter. More specifically, in our <code>ReduceVIF()</code> step, we hard coded two manual criterion in which the algorithm will stop; if the threshold reaches 10, or if the number of features removed hit 20; we can include them in the search space.
</div>

## Retrain on the whole training set

A common practice after the hyperparameter tuning phase is to retrain the model on the whole dataset $X_{\text{train}}$ where we will get the estimator's coefficients obtained from the retraining. This is actually already done as the scikit-learn's `GridSearchCV` has a parameter `refit`; if we select it to be true, then after the model selection process is done (i.e. getting the best hyperparameters after cross validation with grid search), the grid search object will retrain on the whole $X_{\text{train}}$ with the best hyperparameters internally, and return us back an object in which we can call `predict` etc.

### Retrain using optimal hyperparameters

However, to be extra careful, we can retrain manually using the best hyperparameters and check if scikit-learn is true to its documentation. We will just reconstruct the pipeline using the grid's best hyper parameters. We will then test if the retrained model's coefficients coincide with the grid's best estimator's coefficients. If there difference is 0, this means they are trained under the same circumstances and we can be sure that the refit parameter is behaving true to its words.

---

```python
grid_best_hyperparams = grid.best_params_
print(grid_best_hyperparams) ->
{'model__C': 0.3593813663804626,
 'model__penalty': 'l2'}
```

In [ ]:
retrain_pipeline = pipeline.Pipeline(
    [
        ("standardize", preprocessing.StandardScaler()),
        ('remove_multicollinearity', ReduceVIF(thresh=10)),

        (
            "model",
            linear_model.LogisticRegression(
                C=0.3593813663804626, max_iter=10000, random_state=1992, solver="saga", penalty="l1"
            ),
        ),
    ]
)

_ = retrain_pipeline.fit(X_train, y_train)
coef_diff = retrain_pipeline['model'].coef_ - grid.best_estimator_['model'].coef_

print("...")
assert np.all(coef_diff == 0) == True
print("Retraining Assertion Passed!")

## Interpretation of Results

### Interpretation of Coefficients

As shown in the figure below, all else being equal, for every square unit increase in mean cell area, the odds of the tumor being malignant increases by a factor of $e^{1.43} = 4.19$. The variation (standard error) of the characteristics of cells also are deemed important by the model, for example, area se played an important role in determining whether a cell is malignant; intuitively, if some cells are noticably larger than the rest, then it is also a good indicator of malignancy.

In [ ]:
selected_features_by_vif_index = grid.best_estimator_['remove_multicollinearity'].column_indices_kept_ 
selected_feature_names = np.asarray(predictor_cols)[selected_features_by_vif_index]

selected_features_coefficients = grid.best_estimator_['model'].coef_.flatten()

# assertion
#assert grid.best_estimator_['remove_multicollinearity'].feature_names_ == retrain_pipeline['remove_multicollinearity'].feature_names_

fig, ax = plt.subplots(figsize=(15, 15))
# .abs()
_ = pd.Series(selected_features_coefficients, index=selected_feature_names).sort_values().plot(ax=ax, kind='barh')
fig.savefig(config.feature_importance, format="png", dpi=300)

### Interpretation of Metric Scores on Train Set

We are also interested in choosing an optimal threshold for the model such that it gives the lowest recall, or False Negatives. We note that the default threshold when calling `predict()` from a model is $0.5$. In this section, we will explore one way to get the best tradeoff we can when choosing a high recall, while maintaining a reasonable score for precision.

In [ ]:
def evaluate_train_test_set(
    estimator: Callable, X: Union[pd.DataFrame, np.ndarray], y: Union[pd.DataFrame, np.ndarray]
) -> Dict[str, Union[float, np.ndarray]]:
    """This function takes in X and y and returns a dictionary of scores.

    Args:
        estimator (Callable): [description]
        X (Union[pd.DataFrame, np.ndarray]): [description]
        y (Union[pd.DataFrame, np.ndarray]): [description]

    Returns:
        Dict[str, Union[float, np.ndarray]]: [description]
    """

    test_results = {}

    y_pred = estimator.predict(X)
    # This is the probability array of class 1 (malignant)
    y_prob = estimator.predict_proba(X)[:, 1]

    test_brier = metrics.brier_score_loss(y, y_prob)
    test_roc = metrics.roc_auc_score(y, y_prob)

    test_results["brier"] = test_brier
    test_results["roc"] = test_roc
    test_results["y"] = np.asarray(y).flatten()
    test_results["y_pred"] = y_pred.flatten()
    test_results["y_prob"] = y_prob.flatten()

    return test_results


In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    """
    Modified from:
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.89 
    and courtesy of https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65
    """
    plt.figure(figsize=(8, 8))
    plt.title("Precision and Recall Scores as a function of the decision threshold")
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.ylabel("Score")
    plt.xlabel("Decision Threshold")
    plt.legend(loc='best')
    plt.savefig(config.precision_recall_threshold_plot, format="png", dpi=300)
    
def plot_roc_curve(fpr, tpr, label=None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    and courtesy of https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65
    """
    plt.figure(figsize=(8,8))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.legend(loc='best')
    plt.savefig(config.roc_plot, format="png", dpi=300)
    
def adjusted_classes(y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    Will only work for binary classification problems.
    and courtesy of https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65
    """
    return [1 if y >= t else 0 for y in y_scores]

The plots below show the tradeoffs between precision and recall, recall and false positive rate. The confusion matrix on the train set tells us that there is still more false negatives than false positives. We can choose a particular threshold in order to minimize false negatives, at some expense of false positive.

In [ ]:
train_results = evaluate_train_test_set(grid, X_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# CM
cm_train = metrics.confusion_matrix(train_results['y'], train_results['y_pred'])

#### scores
auc = metrics.roc_auc_score(train_results['y'], train_results['y_prob'])

#### annotations
labels = ["True Neg", "False Pos", "False Neg", "True Pos"]
counts = ["{0:0.0f}".format(value) for value in cm_train.flatten()]
percentages = ["{0:.2%}".format(value) for value in cm_train.flatten() / np.sum(cm_train)]

#### final annotations
label = (
    np.array([f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(labels, counts, percentages)])
).reshape(2, 2)

# heatmap
sns.heatmap(
    data=cm_train,
    vmin=0,
    vmax=330,
    cmap=["#fe4a49", "#2ab7ca", "#fed766", "#59981A"],
    linewidth=2,
    linecolor="white",
    square=True,
    ax=ax,
    annot=label,
    fmt="",
    cbar=False,
    annot_kws={"size": 10, "color": "black", "weight": "bold", "alpha": 0.8},
    alpha=1,
)



ax.scatter(1, 1, s=3500, c="white")
ax.text(
    0.72,
    1.0,
    "AUC: {}".format(round(auc, 3)),
    {"size": 10, "color": "black", "weight": "bold"},
)

## ticks and labels
ax.set_xticklabels("")
ax.set_yticklabels("")


## titles and text
fig.text(0, 1.05, "Train Set Confusion Matrix", {"size": 22, "weight": "bold"}, alpha=1)
fig.text(
    0,
    1,
    """Training Set Confusion Matrix.""",
    {"size": 12, "weight": "normal"},
    alpha=0.98,
)


fig.tight_layout(pad=2.5, w_pad=2.5, h_pad=2.5)
fig.savefig(config.final_train_confusion_matrix, format='png', dpi=300)

In [ ]:
# generate the precision recall curve
precision, recall, pr_thresholds = metrics.precision_recall_curve(train_results['y'], train_results['y_prob'])
fpr, tpr, roc_thresholds = metrics.roc_curve(train_results['y'], train_results['y_prob'], pos_label=1)

In [ ]:
# use the same p, r, thresholds that were previously calculated
plot_precision_recall_vs_threshold(precision, recall, pr_thresholds)

Based on the tradeoff plot above, a good threshold can be set at $t = 0.35$, let us see how it performs with this threshold.

In [ ]:
y_pred_adj = adjusted_classes(train_results["y_prob"], t=0.35)

print(
    pd.DataFrame(
        metrics.confusion_matrix(train_results["y"], y_pred_adj),
        columns=["pred_neg", "pred_pos"],
        index=["neg", "pos"],
    )
)

In [ ]:
print(metrics.classification_report(y_true=train_results["y"], y_pred=y_pred_adj))
train_brier = train_results['brier']
print(f"train brier: {train_brier}")

The False Negatives reduced from 15 to 9, at the expense of increase False Positives from 6 to 14. We should take comfort that less patients are falsely classified as benign when in fact they need treatment. This is a tradeoff that we have to choose. The ROC curve below also paints a similar story, in order for you to have high recall, one must accept that there will more False Positives.

In [ ]:
plot_roc_curve(fpr, tpr, 'recall_optimized')

# Evaluation on Test Set

Ultimately, we are interested in finding the estimate of the generalization error of the model, and thus it is time for us to evaluate our model's performance on the "unseen" test set $X_{\text{test}}$ to get a good gauge on how well the model generalizes to unseen data. Take note that now everything has settled, we will use the exact hyperparameters to predict on test set, with the pre-defined threshold of 0.35.

In [ ]:
test_results = evaluate_train_test_set(grid, X_test, y_test)
y_test_pred_adj = adjusted_classes(test_results['y_prob'], t=0.35)

print(pd.DataFrame(metrics.confusion_matrix(test_results['y'], y_test_pred_adj),
                   columns=['pred_neg', 'pred_pos'], 
                   index=['neg', 'pos']))

In [ ]:
test_roc = test_results['roc']
test_brier = test_results['brier']
print(test_roc)
print(test_brier)
print(metrics.classification_report(y_true=test_results["y"], y_pred=y_test_pred_adj))

Using the same threshold we used on training set, we see that the False Negative is quite low. The overall ROC score is 0.9828, and the corresponding Brier Score is 0.04136, both seem reasonably well performing.

# Bias-Variance Tradeoff

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        grid.best_estimator_['model'], X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)

We use the `mlxtend` library to estimate the Bias-Variance Tradeoff in our Logistic Regression model. The core idea behind this function is to use bagging and repeatedly sample from our training set so as to simulate that we are actually drawing samples from the "true" population over a distribution $\mathcal{P}$. 

---

As expected, Logistic Regression being a linear model, its simplicity contributes to its high bias and low variance. 